# Project 2

## Topic : Stereo reconstruction and Non-linear optimization

#### Instructions
<ul>
    <li> The second project of the course is designed to get you familiar with stereo reconstruction, and non-linear optimization </li>
    <li> Use python for this project. PILLOW and OpenCV are permitted for image I/O. </li>
    <li> Submit this notebook as a zipped file on moodle. The format should be $<$team_id$>$_$<$team_ name$>$.zip. Both members have to submit this zip file. </li>
    <li> A seperate report is not needed if you're coding in the notebook itself. Please provide adequate descriptions of the approaches you've taken. Also mention work distribution for the two members. </li>
    <li> Refer to the late day policy. Start early </li> 
    <li> Download data from here: https://iiitaphyd-my.sharepoint.com/:f:/g/personal/aryan_sakaria_students_iiit_ac_in/Er5C7351IAlFsvwHUesFeSQBQtlSiAS7AORSEJT2qH_8_w?e=ol98k9  </li>
</ul>

----
### PART 1: Stereo dense reconstruction

3-D point clouds are very useful in robotics for several tasks such as object detection, motion estimation (3D-3D matching or 3D-2D matching), SLAM, and other forms of scene understanding.  Stereo camerasprovide  us  with  a  convenient  way  to  generate  dense  point  clouds.Densehere,  in  contrast  tosparse,means all the image points are used for the reconstruction.  In this part of the assignment you will begenerating a dense 3D point cloud reconstruction of a scene from stereo images.

#### Procedure: 

<ol>
    <li> Generate a disparity map for each stereo pair.  Use OpenCV (e.g.  StereoSGBM) for this.  Notethat the images provided are already rectified and undistorted. </li>
    <li> Then, using the camera parameters and baseline information generate colored point clouds fromeach disparity map.  Some points will have invalid disparity values, so ignore them.  Use [Open3D]for storing your point clouds. </li>
    <li> Register (or transform) all the generated point clouds into your world frame by using the providedground truth poses. </li>
    <li> Visualize the registered point cloud data, in color.  Use Open3D for this </li>
</ol>
    

In [1]:
#import libraries:
import numpy as np 
import cv2
import open3d as o3d
import matplotlib.pyplot as plt

In [6]:
def read_transformations(filename='poses.txt'):
    f = open(filename, 'r')
    lines = f.readlines()
    transformation_list = []
    for i in range(len(lines)):
        transformation_list_temp = lines[i].split()
        temp_rot = [] 
        temp_rot.append( (transformation_list_temp[0:4] ) ) 
        temp_rot.append( (transformation_list_temp[4:8]  ) ) 
        temp_rot.append( (transformation_list_temp[8:12]  ) ) 
        transformation_list.append(np.array(temp_rot).astype('float64'))
    return transformation_list

In [7]:
def read_calib(filename='calib.txt'):
    f = open(filename, 'r')
    lines = f.readlines()
    K = np.reshape(np.array(np.float64(lines[1].split())),[3,3])
    b = np.array(np.float64(lines[4].split()[0]))
    return K,b

In [8]:
def getDisparity(imgL, imgR):
    
    imgL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
    imgR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)
    stereo = cv2.StereoSGBM_create(minDisparity = -35,
        numDisparities = 144,
        disp12MaxDiff = 1,
        blockSize=5,
        P1=8 * 3 * 5 ** 2,    
        P2=32 * 3 * 5 ** 2,
        uniquenessRatio = 10,
        speckleWindowSize = 100,
        speckleRange = 32,
        preFilterCap=63
        )
    disparity = stereo.compute(image_left, image_right).astype(np.float32) / 16.0
    disparity = (disparity+35)/140
    return disparity

In [9]:
def project_points(points, colors, K, P, width, height):
    
    R = P[:, :3]
    R[:, 1] *= -1
    t = P[:, 3]
    
    dist_coeff = np.zeros((4, 1))
    projected, _ = cv2.projectPoints(points, R, t, K, dist_coeff)

    ones_row = np.ones((points.shape[0], 1))

    
    points = np.concatenate((points, ones_row), axis = 1)
    
    camera_projected = P @ points.T
    camera_projected = camera_projected.T.astype(np.int)
    
    xy = projected.reshape(-1, 2). astype(np.int)
    mask = (
        (0 <= xy[:, 0]) & (xy[:, 0] < width) &
        (0 <= xy[:, 1]) & (xy[:, 1] < height)
    )
    
    img_coord = xy[mask]
    img_color = colors[mask]
    
    camera_color = colors
    
    return camera_projected, camera_color, img_coord, img_color

In [10]:
def get3Dptstolist(disparity):
    points_list = []
    x, y = disparity.shape
    
    for i in range(x):
        for j in range(y):
            points_list.append([j, i, disparity[i, j], 1])
            
    return np.array(points_list)

In [11]:
def reprojectImageTo3D(Q, points_mat):
    points_3D = Q @ points_mat.T
    points_3D /= points_3D[3, :]
    points_3D = points_3D[:3, :]
    points_3D = points_3D.T
    
    output = np.zeros((370, 1226, 3))
    i = 0
    j = 0
    
    for p in range(points_3D.shape[0]):
        output[i][j] = points_3D[p]
        j += 1
        
        if j >= 1226:
            j = 0
            i += 1
            
    return output

In [12]:
def getImage(points, color, height, width):
    image = np.zeros((height, width, 3), dtype=color.dtype)
    image[points[:, 1], points[:, 0]] = color
    return image

In [13]:
K,b=read_calib()
poses = read_transformations()

Q = np.array([
        [1, 0,    0, -K[0,2]],
        [0, 1,    0, -K[1,2]],
        [0, 0,    0,   K[0,0]],
        [0, 0, -1/b,   0]
    ])

all_pts = np.array([]).reshape(0,3)
all_colors = np.array([]).reshape(0,3)
for i in range(460, 481):
    image_left = cv2.imread('img2/0000000{}.png'.format(i))
    image_right = cv2.imread('img3/0000000{}.png'.format(i))

    disparity = getDisparity(image_left, image_right)
    points_list = get3Dptstolist(disparity)
    points_3D = reprojectImageTo3D(Q, points_list)

    mask_map = disparity > disparity.min()

    colors = cv2.cvtColor(image_left, cv2.COLOR_BGR2RGB)

    output_points = points_3D[mask_map]
    output_colors = colors[mask_map]
    
    camera_projected, camera_color, img_coords, img_color = project_points(output_points, output_colors, K, poses[i-460], disparity.shape[1], disparity.shape[0])
    
    all_pts = np.concatenate((all_pts, camera_projected), axis=0)
    all_colors = np.concatenate((all_colors, camera_color), axis=0)
    
    img = getImage(img_coords, img_color, disparity.shape[0], disparity.shape[1])
    cv2.imwrite("./result/img" + str(i) + ".png", cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

<ipython-input-11-40ae8300b214>:3: RuntimeWarning: divide by zero encountered in true_divide
  points_3D /= points_3D[3, :]
<ipython-input-11-40ae8300b214>:3: RuntimeWarning: invalid value encountered in true_divide
  points_3D /= points_3D[3, :]
<ipython-input-9-790032059ad8>:15: RuntimeWarning: invalid value encountered in matmul
  camera_projected = P @ points.T


In [14]:
mask = (
        (-3000 <= all_pts[:, 0]) & (all_pts[:, 0] < 3000) &
        (-3000 <= all_pts[:, 1]) & (all_pts[:, 1] < 3000) & 
        (-900 <= all_pts[:, 2]) & (all_pts[:, 2] < 900)
    )

plot_pts = all_pts[mask]
plot_pts[:, 1] *= -1
plot_colors = (all_colors[mask]/255.0).astype(float)

pcd = o3d.geometry.PointCloud()
vis = o3d.visualization.Visualizer()
pcd.points = o3d.utility.Vector3dVector(plot_pts) 
pcd.colors = o3d.utility.Vector3dVector(plot_colors) 

o3d.visualization.draw_geometries([pcd])

# Problem setup

Our goal will be to visualize the depth of objects found in a set of stereo images. Essentially we will produce a gray scale heat map whereby lighter shades of gray will signify objects close to the camera lens with progressively darker shades to distinguish objects further away.

Disparity Maps
--------------

We will get depth information from our stereo images using Stereo Triangulation. The depth of a point in real world space can be calculated through the following.
Note the optical centers of both our cameras at O and O’ in addition to the point in 3D space whose depth we are trying to find; point X. Length f is the focal length of our cameras which we know from our intrinsic values found in the first part of this series. Now x and x’ are the points on the 2D image plane where point X will appear in each image respectively. The idea here is that the equivalent triangles as denoted Ofx and O’fx’ can be compared to each other in order to extract the depth Z of point X. The equation that allows us to accomplish this is shown below.
$$disparity = x - x' = \frac{Bf}{Z}$$
<center>
    <img src="./theory_image/img1.jpeg">
</center>
</br>
</br>
If we try to do this using homogeneous coordinate representation, we will get the following: <br>
<br>
$\begin{equation}\begin{bmatrix}U \\ V \\ W \\ T\end{bmatrix} = \begin{bmatrix}B & 0 & 0 & -c_{x}\\ 0 & B & 0 & -c_{y} \\ 0 & 0 & B\cdot c & 0 \\ 0 & 0 & 0 & 1\\\end{bmatrix}\cdot\begin{bmatrix}x' \\ y' \\ 1 \\ p_{x}\end{bmatrix}\end{equation}$</br>

which implies

$\begin{equation}\begin{bmatrix}U \\ V \\ W \\ T\end{bmatrix} = \begin{bmatrix}1 & 0 & 0 & -c_{x}\\ 0 & 1 & 0 & -c_{y} \\ 0 & 0 & c & 0 \\ 0 & 0 & -1/B & 0\\\end{bmatrix}\cdot\begin{bmatrix}x' \\ y' \\ p_{x} \\ 1\end{bmatrix}\end{equation}$

Epipolar Geometry
-----------------
Consider a scenario with two cameras taking a picture of the same scene. 
<center>
    <img src="./theory_image/img2.jpeg">
</center>
</br>
</br>
By examining the line OX pictured above that is emanating from the center-point of the left camera we can see that there are 3 points along that line in which we want to know their distance from the point O. As stated early we’re not going to be able to extract this information from the left sided camera alone. With the right sided camera however we can see that the 3 points of X that are in question can be projected along a single line on the plane of its image (the line denoted with: l’). This line is referred to as the epiline; the line in which any point X along the line OX must appear. It’s significance is that when we are searching for the matching point of X in the right sided camera’s image we already know that it will appear along this epiline therefore greatly reducing our search effort. Furthermore we can assume every point that appears in the left handed image will always have an accompanying epiline found in the right sided image. This is known as the epipolar constraint.

The epipole is the line that connects the center-points of each camera. In the above figure the epiline is the segment 00‘. The points e and e’ are the points in which the center-point of the opposing camera will appear in its counterpart’s image. By including the point X(the point whose depth value we are trying to extract) with the epipole line we can derive the plane denoted as XOO'. This plane is referred to as the epipolar plane.

The essential matrix contains information about translation and rotation which describes the camera’s position relative to its counterpart. 
But for our solution we want to make our calculations in terms of camera pixel coordinates. It contains the same information as the essential matrix but also includes intrinsic information about both cameras so that we can relate the two in such terms. What the fundamental matrix allows us to do is map a single point in one image to its corresponding epiline in the other giving us a starting point to later find the epipole line between the two cameras center points and the epipolar plane.

SGBM and Disparity Map
----------------------
The Semi-Global Block Matching (SGBM) method is based on the idea of pixelwise matching of Mutual Information and approximating a global, 2D smoothness constraint by combining many 1D constraints. Pixelwise Matching Cost Calculation is done by the sampling insensitive measure of Birchfield and Tomasi. The cost CBT (p, d) is calculated as the absolute minimum difference of intensities at p and q = ebm(p, d) in the range of half a pixel in each direction along the epipolar line. Pixelwise cost calculation is generally ambiguous and wrong matches can easily have a lower cost than correct ones, due to noise, etc. Therefore, an additional constraint is added that supports smoothness by penalizing changes of neighboring disparities. The pixelwise cost and the smoothness constraints are expressed by defining the energy E(D) that depends on the disparity image D. 

The problem of stereo matching can now be formulated as finding the disparity image D that minimizes the energy E(D). This leads to the new idea of aggregating matching costs in 1D from all directions equally. The aggregated (smoothed) cost S(p, d) for a pixel p and disparity d is calculated by summing the costs of all 1D minimum cost paths that end in pixel p at disparity d. The disparity image $D_{b}$ that corresponds to the base image Ib is determined as in local stereo methods by selecting for each pixel p the disparity d that corresponds to the minimum cost, i.e. min(d) over S(p, d).

The resulting disparity image can still contain certain kinds of errors. Furthermore, there are generally areas of invalid values that need to be recovered. Both can be handled by post processing of the disparity image.

----
### PART 2: Motion estimation using iterative PnP

Using the generated reconstruction from the previous part, synthesize a new image taken by a virtualmonocular camera fixed at any arbitrary position and orientation.  Your task in this part is to recoverthis pose using an iterative Perspective-from-n-Points (PnP) algorithm. 

#### Procedure: 

<ol>
    <li> Obtain a set of 2D-3D correspondences between the the image and the point cloud.  Since hereyou’re generating the image, this should be easy to obtain. </li>
    <li> For this set of correspondences compute the total reprojection error c= $\sum_{i} ‖x_i−P_{k}X_i‖^2 $    where $P_{k}= K[R_{k}|t_{k}]$, $X_{i}$ is the 3D point in the world frame, $x_{i}$ is its corresponding projection. </li>
    <li> Solve for the pose $T_{k}$ that minimizes this non-linear reprojection error using a Gauss-Newton (GN)scheme.  Recall that in GN we start with some initial estimated value $x_{o}$ and iteratively refine the estimate using $x_{1}$= $∆x+x_0$, where $∆x$ is obtained by solving the normal equations $J^{T}J∆x$= -$J^{T}e$, until convergence.The main steps in this scheme are computing the corresponding Jacobians and updating the estimates correctly.  For our problem,  use a 12×1 vector parameterization for $T_{k}$(the top 3×4submatrix).  Run the optimization for different choices of initialization and report your observations. </li>
</ol>

In [15]:
def DLT(x,X):
    M=[]
    for i in range(X.shape[0]):
        Xi=X[i][0]
        Yi=X[i][1]        
        Zi=X[i][2]
        xi=x[i][0]
        yi=x[i][1] 
        axi=np.array([-Xi,-Yi,-Zi,-1,0,0,0,0,xi*Xi,xi*Yi,xi*Zi,xi])
        ayi=np.array([0,0,0,0,-Xi,-Yi,-Zi,-1,yi*Xi,yi*Yi,yi*Zi,yi])  
        M.append(axi)
        M.append(ayi)
    u, s, v = np.linalg.svd(M, full_matrices=False)
    p=v[11].reshape((3,4))
    p=p/p[2][3]
    return p

def jacob(X, K, P):
    J=[]
    X=np.insert(X,3,1,axis=1)
    proj_X=P@X.T
    proj_X=proj_X.T
    for i in range(X.shape[0]):
        Xi=X[i][0]
        Yi=X[i][1]        
        Zi=X[i][2]
        xi=proj_X[i][0]
        yi=proj_X[i][1]
        zi=proj_X[i][2]
        jxi=np.array([Xi/zi,Yi/zi,Zi/zi,1/zi,0,0,0,0,-xi*Xi/(zi**2),-xi*Yi/(zi**2),-xi*Zi/(zi**2),-xi/(zi**2)])
        jyi=np.array([0,0,0,0,Xi/zi,Yi/zi,Zi/zi,1/zi,-yi*Xi/(zi**2),-yi*Yi/(zi**2),-yi*Zi/(zi**2),-yi/(zi**2)])
        J.append(jxi*K[0,0])
        J.append(jyi*K[1,1])
    return np.array(J)

def error(x,pred_x):
    diff=x-pred_x
    diff=diff.reshape(-1)
    return diff@diff

def gaussNewtonForPose(x,X,K,P,num_epoch,lr):
    X=np.insert(X,3,1,axis=1)
    for epoch in range(num_epoch):
        pred_x=K@P@X.T
        pred_x=pred_x.T
        pred_x=pred_x/pred_x[:,2].reshape(-1,1)
        pred_x=pred_x[:,:2]
        error_vec=x-pred_x
        error_vec=error_vec.reshape(-1)
        print(f'Epoch {epoch} error is {error(x,pred_x)}')
        J=jacob(X[:,:3],K,P)
        delta_P=np.linalg.pinv(J.T@J)@J.T@error_vec #(J.T*J)*delta = -J.T*error
        P+=lr*delta_P.reshape(3,4)
    return P

In [16]:
im_num = 2
im_num += 460
image_left = cv2.imread('img2/0000000{}.png'.format(im_num))
image_right = cv2.imread('img3/0000000{}.png'.format(im_num))
disparity = getDisparity(image_left, image_right)
points_list = get3Dptstolist(disparity)
points_3D = reprojectImageTo3D(Q, points_list).reshape(-1,3)
mask_map = disparity.ravel() > disparity.min()
output_points = points_3D[mask_map]
im_points =points_list[mask_map][:,:2]

<ipython-input-11-40ae8300b214>:3: RuntimeWarning: divide by zero encountered in true_divide
  points_3D /= points_3D[3, :]
<ipython-input-11-40ae8300b214>:3: RuntimeWarning: invalid value encountered in true_divide
  points_3D /= points_3D[3, :]


In [17]:
P = DLT(im_points[50:305],output_points[50:305])
_ = gaussNewtonForPose(im_points[300:3000],output_points[300:3000],K,P,50,1)

Epoch 0 error is 587249231886459.8
Epoch 1 error is 146774373188396.03
Epoch 2 error is 36674443537164.72
Epoch 3 error is 9158854160970.219
Epoch 4 error is 2284655096621.805
Epoch 5 error is 568458268998.6702
Epoch 6 error is 140592985680.53595
Epoch 7 error is 34233239885.835976
Epoch 8 error is 7974583887.77809
Epoch 9 error is 1626090948.9628906
Epoch 10 error is 221397950.45647654
Epoch 11 error is 9787559.830917489
Epoch 12 error is 31215.633279083588
Epoch 13 error is 0.28539021718359436
Epoch 14 error is 1.9933721216796735e-11
Epoch 15 error is 1.7951169318820446e-23
Epoch 16 error is 1.5977356111165867e-23
Epoch 17 error is 6.208891502197131e-23
Epoch 18 error is 2.92822318384529e-23
Epoch 19 error is 5.432393486101127e-23
Epoch 20 error is 1.9710535938688355e-23
Epoch 21 error is 2.3713869339158114e-23
Epoch 22 error is 3.10575923855353e-23
Epoch 23 error is 2.2841567999795462e-23
Epoch 24 error is 3.5693907132342926e-23
Epoch 25 error is 6.237015098857473e-23
Epoch 26 error

In [19]:
## Try different Initialisations
P = np.random.normal(0,1,(3,4)) # Random Init
_ = gaussNewtonForPose(im_points[300:3000],output_points[300:3000],K,P,50,1)

Epoch 0 error is 36295027039.72218
Epoch 1 error is 37496445761.98317
Epoch 2 error is 36315689460.9054
Epoch 3 error is 37578787326.360146
Epoch 4 error is 36337349917.85321
Epoch 5 error is 37686265776.263565
Epoch 6 error is 36360746182.71019
Epoch 7 error is 37836973960.32866
Epoch 8 error is 36387212388.206024
Epoch 9 error is 38074617447.44865
Epoch 10 error is 36419708568.40748
Epoch 11 error is 38545427000.87638
Epoch 12 error is 36467631026.87027
Epoch 13 error is 40350264770.71357
Epoch 14 error is 36609404584.36476
Epoch 15 error is 34371920727.36791
Epoch 16 error is 36669809975.026344
Epoch 17 error is 35214331010.82478
Epoch 18 error is 36697512685.26453
Epoch 19 error is 35402330609.0472
Epoch 20 error is 36720732833.39656
Epoch 21 error is 35518286635.03375
Epoch 22 error is 36741846555.83659
Epoch 23 error is 35601673546.02928
Epoch 24 error is 36761838228.555786
Epoch 25 error is 35666552681.367386
Epoch 26 error is 36781253864.45174
Epoch 27 error is 35719570641.7044

In [19]:
P = DLT(im_points[100:400],output_points[100:400]) #Different range of image points for DLT
_ = gaussNewtonForPose(im_points[300:3000],output_points[300:3000],K,P,50,1)

Epoch 0 error is 597418815360151.9
Epoch 1 error is 149333020042048.22
Epoch 2 error is 37322380559974.734
Epoch 3 error is 9325124968427.148
Epoch 4 error is 2328513464100.3984
Epoch 5 error is 580711954698.7218
Epoch 6 error is 144437626640.30023
Epoch 7 error is 35707822931.279366
Epoch 8 error is 8696341093.120365
Epoch 9 error is 2032378405.905208
Epoch 10 error is 418428233.86373514
Epoch 11 error is 56168985.029705375
Epoch 12 error is 1704251.381394114
Epoch 13 error is 788.1011704808392
Epoch 14 error is 0.0001063355870836464
Epoch 15 error is 1.905301271546484e-18
Epoch 16 error is 3.254459474003103e-23
Epoch 17 error is 5.1846400990841944e-23
Epoch 18 error is 6.653835597472134e-23
Epoch 19 error is 3.027607074641146e-23
Epoch 20 error is 9.882229881700032e-24
Epoch 21 error is 2.2219997561704986e-23
Epoch 22 error is 3.370785944547458e-23
Epoch 23 error is 3.6354554973617316e-23
Epoch 24 error is 1.692823443347288e-23
Epoch 25 error is 1.3611483633050706e-23
Epoch 26 error 

## Observations

* For Random initialisation, the error plateaus earlier than a DLT Based initialisation.
* For different points in the image, the performance differs slightly indicating that if we use a better method to select points for DLT, maybe we will get better performance.

# Project 2 - Part B:

## 1. SfM pipeline (`6 mark`)

To get the context of below questions, take a look at the code above: The same questions have been asked at different places above as comments in the code.

1. `0.5 mark` **Basics** - How do we know this (`camera_ind`) information in practical setting? In other words, how do we know observations in `points_2d` belong to which camera. Explain. 
    - Ans-1 - Basics: If we consider the observations in `points_2d` they must have been acquired when a 3D observation was projected onto the camera plane. If we follow this logic, any 3D point can be imaged in a 2D plane. There could be multiple 2D points which satisfy this which we can verify by comparing with the queried 2D point using `points_ind`.
    
2. `0.5 mark` **Basics** - How do we know this (`point_ind`) information in practical setting?  In other words, how do we know observations in `points_2d` belong to which 3D point. Explain.
    - Ans-2 - Basics: We are given camera parameters(both extrinsics and intrinsics). We can use this to reproject 3D points into 2D points. The camera index registration provided in `camera_ind` helps us to perform point wise registration to get `points_ind`. 
    
3. `0.5 mark` **Transformations** - `project()` function: In the `project()` function, would it make any difference if I do translate first, then rotate? Why/why not?
    - Ans-3 - Transformations: In the given `project()` function, first it is rotated and then translated because the translation should happen around the original axes of the camera frame. If we performed a translation first then the rotation is with respect to the image plane. This would give different results (from linear algebra, we know order of matrix multiplication matters)

4. `0.5 mark` **Jacobian** - `bundle_adjustment_sparsity()` function: m above is not "M*N" (*2) unlike our lecture notes. Why is that so?
    - Ans-4 - Jacobian: In the class, the Jacobian shown was a set of derivatives corresponding to each observation vs the camera parameters and world points. So the total number of row entries will maximally be $(number\;of\;cameras\;\cdot\;number\;of\;points)$ which is $2MN$. But $m$ is given as length of `camera_ind` and this is the optimal number of row entries in the function given. This is because some observations will not have corresponding camera projections which will make the entire row entry 0. So we can remove this row and other similar rows leaving us with rows with unique camera registrations which is equal to length of `camera_ind`.

5. `2 mark` **Jacobian & Parameters** - `bundle_adjustment_sparsity()` function: 
    1.  Why are we doing `n_cameras * 9` here instead of `n_cameras * 12`? Recollect: Every individual motion Jacobian was (1*)12 in our lecture notes. 
        - Ans 5.1 - Jacobian & Parameters: In the given function the extrinsic camera parameters have the rotation specified in its vector form but in class the parameters used have the matrix representation. 
        
    2. Ignoring the scale parameters, what was the number of unknown parameters in our lecture notes in terms of `n_cameras` and `n_points`? What is it here in the code? Is it different? If so, what is and why? [Link of notes](https://www.notion.so/Stereo-Structure-from-Motion-9fdd81e4194f4803ac9ba7552df56470).
        - Ans 5.2 - Jacobian & Parameters: In the lecture, we had 12 camera parameters for each of $M$ cameras and 3 coordinates for each of the $N$ observations giving us $12M+3N$ optimization variables. However in the given code, the number of optimization variables are $9m + 3n$. This is because of the sparse representation of `scipy.sparse.lil_matrix` used in the given function which removed the zero-entry rows mentioned in part 4.

6. `2 mark` **Sparsity, Residual Vector & Jacobian** - `bundle_adjustment_sparsity()` function: Explain what you understand from above 6 lines of code by coding a simple toy example yourself to illustrate how it is different from what you've learnt in class. ([Coding toy example + elaborating in words]- both are compulsory.)
    - Ans 6 - The jacobian matrix in the given function contains only the non zero rows such that the memory allocation is optimized(In the normal representation, the code was asking 1.5 TB of memory). If we check how the jacobian looks, we can see that for an observation, the camera parameters are non zero for the corresponding camera index. Similarly, we can see that the coordinates of 3D are non zero for the corresponding point index, so the other zero values are removed. In comparison to the class example, the jacobian has neighbouring zeros in these locations.

## 2. Initializing R,t and 3D points for SfM given 2 images (`4 mark`)

Using OpenCV functions, mention how you would initialize R,t (poses) and 3D points for SfM given 2 images and K matrix. You don't need to implement it, just mention function names with input/output arguments clearly and briefly explain what they do (You don't need to give detailed answers). A sample answer could be as follows:

**Ans 2:**

1. First, we do feature matching using the following function

    `function_name()`: It takes ____ (images?) as input arguments and gives us ____ outputs. These outputs are passed to the next step.

2. ...
3. Now that we have essential matrix, we can decompose it into R,t as following:

    `Rt = cv2.recoverPose(E, pts0, pts1, K)`: This takes Essential matrix, K matrix and the corresponding points in 2 images as input arguments and gives us R,t. `Rt[1]` would give us R and `Rt[2]` would give us t.

4. Now that we have ____, we find 3D points as follows: 

### Algorithm

SFM has a couple of important steps which involve finding point correspondences, followed by calculating the fundamental matrix to describe the epipolar geometry constraints. We now find the orientation and location of second camera in the coordinate system of the first camera. This helps us find the baseline. The 3D locations are then calculated by stereo triangulation.

1. The point correspondences can be found in two ways: one by feature matching for which we can use SURF Features found to `cv2.SURF(HessianThreshold).detectAndCompute(img)` which can be matched using a matcher algorithm like Brute force matcher or `cv2.FlannBasedMatcher` (FLANN). Alternatively we can do feature tracking by Optical Flow using Lucas-Kanade.
2. Fundamental Matrix is calculated by `cv2.findFundamentalMat`
3. `cv2.recoverPose` recover relative camera rotation and translation from an estimated essential matrix and the corresponding points in two images, using cheirality check.
4. `cv2.triangulatePoints` helps to perform stereo triangulation.
    

